# Testing models

In [1]:
import sys
sys.path.append('src/')

from Models import deberta_nli, bart_nli

# create helper function to standardise order


## Label structure

deberta: ['contradiction', 'entailment', 'neutral']

bart: ['contradiction', 'neutral', 'entailment']


In [6]:
deberta_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

([['0.96940809', '0.03018436', '0.00040754']], ['contradiction'])

In [7]:
bart_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

([['0.93711895', '0.05809933', '0.00478182']], ['contradiction'])